In [44]:
import pandas as pd
import numpy as np
import reverse_geocoder as rg
import matplotlib.pyplot as plt

In [40]:
allmodis = pd.read_csv(r'..\Data\Raw\archive_M6.csv').drop(['acq_time', 'satellite', 'instrument', 'version', 'daynight', 'type'], 1)
allviirs = pd.read_csv(r'..\Data\Raw\archive_V1.csv').drop(['acq_time', 'satellite', 'instrument', 'version', 'type'], 1)

In [9]:
allmodis.head()

,latitude,longitude,brightness,scan,track,acq_date,confidence,bright_t31,frp
0,-16.4439,143.5076,319.9,4.5,2.0,2000-11-01,59,300.6,62.8
1,-16.4447,143.5152,321.0,4.5,2.0,2000-11-01,62,300.5,71.6
2,-18.4459,144.8904,324.6,3.2,1.7,2000-11-01,74,303.1,77.4
3,-18.4401,144.8603,317.7,3.3,1.7,2000-11-01,61,303.2,38.5
4,-18.6804,145.5470,317.3,2.9,1.6,2000-11-01,54,299.4,32.8


In [5]:
allmodis.to_csv(r'..\Data\Clean\carchive_M6.csv', index = False)
allviirs.to_csv(r'..\Data\Clean\carchive_V1.csv', index = False)

In [3]:
nrtmodis = pd.read_csv(r'..\Data\Raw\nrt_M6.csv').drop(['acq_time', 'satellite', 'instrument', 'version', 'daynight'], 1)
nrtviirs = pd.read_csv(r'..\Data\Raw\nrt_V1.csv').drop(['acq_time', 'satellite', 'instrument', 'version', 'daynight'], 1)

In [9]:
nrtmodis.to_csv(r'..\Data\Clean\cnrt_M6.csv', index = False)
nrtviirs.to_csv(r'..\Data\Clean\cnrt_V1.csv', index = False)

In [15]:
rain = pd.read_csv(r'..\Data\Raw\rain.csv')

In [56]:
rain.head()

,Rain ID,Date/Time,Nearest town,State,Latitude,Longitude,Severity confidence description,Intense precipitation amount,Intense precipitation period,Total precipitation,Total precipitation duration,Water course,Damage total,Insurance payout,Number of deaths,Number of injuries,Unnamed: 16
0,989,2006-01-06 05:05:00,walcha,NSW,-30.93,151.72,Estimate of rainfall (eg. from filling of rece...,25.0,10.0,35,0,NaN,0.0,0.0,0.0,0.0,NaN
1,990,2006-01-07 01:00:00,rosebank,NSW,-28.64,153.41,Other measurement of rainfall.,0.0,0.0,505,0,NaN,0.0,0.0,0.0,0.0,NaN
2,991,2006-01-12 05:15:00,Walcha,NSW,-30.99,151.59,Other measurement of rainfall.,54.0,35.0,54,0,NaN,0.0,0.0,0.0,0.0,NaN
3,992,2006-01-15 05:20:00,Forbes,NSW,-33.39,148.01,Other measurement of rainfall.,53.0,60.0,53,0,NaN,0.0,0.0,0.0,0.0,NaN
4,993,2006-01-15 21:54:00,Goulburn,NSW,-34.75,149.73,Bureau measurement of rainfall.,85.0,240.0,85,0,NaN,0.0,0.0,0.0,0.0,NaN


In [ ]:
rain.nunique()

In [16]:
rain['Date/Time'] = pd.to_datetime(rain['Date/Time'])

In [17]:
rain['acq_date'] = [i.date() for i in rain['Date/Time']]
rain['acq_time'] = [i.time() for i in rain['Date/Time']]

In [18]:
rain = rain.drop(['Unnamed: 16', 'Severity confidence description', 'Water course', 'Number of injuries', 'Number of deaths', 'Damage total', 'Damage total', 'Date/Time'], 1)

In [57]:
rain['region'] = [i['cc'] for i in rg.search(list(zip(*[np.array(rain.Latitude), np.array(rain.Longitude)])))]

In [58]:
rain['region'].unique()

array(['AU', 'GQ', 'GH', 'CX'], dtype=object)

In [56]:
rg.search((rain.Latitude[0], rain.Longitude[0]))

[OrderedDict([('lat', '-30.73333'),
              ('lon', '151.8'),
              ('name', 'Enmore'),
              ('admin1', 'New South Wales'),
              ('admin2', 'Armidale Dumaresq'),
              ('cc', 'AU')])]

In [47]:
import folium
import folium.plugins as plugins
f = folium.Figure(width=750, height=500)
center_lat = -24.003249 
center_long = 133.737310
m = folium.Map(location=[center_lat,center_long], control_scale=True, zoom_start=4,width=750, height=500,zoom_control=True).add_to(f)
for i in range(0,rain.shape[0]):    
    location=[rain.iloc[i]['Latitude'], rain.iloc[i]['Longitude']]
    folium.CircleMarker(location,radius=1,color='red').add_to(m)
#data_heat = df2[['latitude','longitude','frp']].values.tolist()
#plugins.HeatMap(data_heat).add_to(m)
display(m)

In [60]:
rain.to_csv(r'..\Data\Clean\rain.csv', index = False)

In [61]:
wind = pd.read_csv(r'..\Data\Raw\wind.csv')

In [62]:
wind.head()

,Wind ID,Date/Time,Duration,Latitude,Longitude,Nearest town,State,Source,Max Gust speed,Max Gust direction,...,Max Mean Wind direction,Path length,Path width,Path direction,Fujita scale,Damage total,Insurance payout,Number of deaths,Number of injuries,Unnamed: 20
0,1376,2006-01-01 03:32:00,0,-36.88,149.24,Bombala,NSW,AWS,48,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN
1,1377,2006-01-01 04:44:00,0,-35.94,148.38,Cabramurra,NSW,AWS,63,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN
2,1378,2006-01-01 05:16:00,0,-36.23,149.12,Cooma,NSW,AWS,56,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN
3,1379,2006-01-06 04:30:00,0,-30.93,151.72,walcha,NSW,SPOTTER,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN
4,1380,2006-01-06 08:45:00,0,-28.85,153.05,Casino,NSW,AWS,54,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN


In [15]:
wind.nunique()

Wind ID                    2833
Date/Time                  2715
Duration                     61
Latitude                   1246
Longitude                  1258
Nearest town               1146
State                         7
Source                      461
Max Gust speed               76
Max Gust direction          267
Max Mean Wind speed          45
Max Mean Wind direction      46
Path length                   7
Path width                    7
Path direction                8
Fujita scale                  3
Damage total                  3
Insurance payout              1
Number of deaths              2
Number of injuries            2
Unnamed: 20                   0
dtype: int64

In [63]:
wind['Date/Time'] = pd.to_datetime(wind['Date/Time'])

In [64]:
wind['acq_date'] = [i.date() for i in wind['Date/Time']]
wind['acq_time'] = [i.time() for i in wind['Date/Time']]

In [65]:
wind = wind.drop(['Unnamed: 20', 'Number of injuries', 'Number of deaths', 'Insurance payout', 'Damage total', 'Date/Time'], 1)

In [66]:
wind.to_csv(r'..\Data\Clean\wind.csv', index = False)

In [67]:
lightning = pd.read_csv(r'..\Data\Raw\lightning.csv')

In [36]:
lightning.head()

,Lightning ID,Date/Time,Latitude,Longitude,Nearest town,State,Source,Severity confidence description,Max current,Flash rate,Damage total,Insurance payout,Number of deaths,Number of injuries,Power line deaths
0,242,2006-03-01 00:00:00,-35.0180,117.8840,Albany,WA,Albany Extra,NaN,0,0,0.0,0.0,0.0,0.0,0.0
1,266,2009-10-05 04:00:00,-34.5067,150.7929,DAPTO,NSW,NaN,based only on physical evidence at the scene a...,0,0,NaN,NaN,NaN,NaN,NaN
2,267,2009-11-17 07:00:00,-28.6651,153.2845,THE CHANNON,NSW,Spotter,based only on physical evidence at the scene a...,0,0,NaN,NaN,NaN,NaN,NaN
3,269,2011-03-11 03:40:00,-31.1095,150.9147,SOUTH TAMWORTH,NSW,Storm Spotter,based on actual observation of event by other ...,0,0,NaN,NaN,NaN,NaN,NaN
4,270,2011-02-28 08:15:00,-31.9652,141.4513,BROKEN HILL,NSW,Storm Spotter,based on actual observation of event by other ...,0,0,NaN,NaN,NaN,NaN,NaN


In [24]:
lightning.nunique()

Lightning ID                       20
Date/Time                          19
Latitude                           18
Longitude                          18
Nearest town                       18
State                               5
Source                              7
Severity confidence description     3
Max current                         1
Flash rate                          3
Damage total                        1
Insurance payout                    1
Number of deaths                    1
Number of injuries                  1
Power line deaths                   1
Unnamed: 15                         0
dtype: int64

In [68]:
lightning['Date/Time'] = pd.to_datetime(lightning['Date/Time'])

In [69]:
lightning['acq_date'] = [i.date() for i in lightning['Date/Time']]
lightning['acq_time'] = [i.time() for i in lightning['Date/Time']]

In [70]:
lightning = lightning.drop(['Unnamed: 15', 'Power line deaths', 'Date/Time', 'Number of injuries', 'Number of deaths', 'Insurance payout', 'Damage total', 'Flash rate', 'Max current', 'Severity confidence description'], 1)

In [71]:
lightning.to_csv(r'..\Data\Clean\lightning.csv', index = False)

In [6]:
nrtmodis['acq_date'] = pd.to_datetime(nrtmodis['acq_date'])
df = nrtmodis.groupby([pd.Grouper(key='acq_date', freq='M')])['brightness'].mean().reset_index().sort_values('acq_date')

In [11]:
df = nrtmodis.groupby([pd.Grouper(key='acq_date', freq='M')])['brightness'].mean().reset_index().sort_values('acq_date')
#bla = nrtmodis.groupby('acq_date',pd.Grouper(freq = 'M')).mean()

In [12]:
df

,acq_date,brightness
0,2020-01-31,341.666352
1,2020-02-29,339.083109
2,2020-03-31,326.820262
3,2020-04-30,325.316150
4,2020-05-31,323.824857


In [39]:
list(zip(*[np.array(rain.Latitude), np.array(rain.Longitude)]))

[(-30.93, 151.72),
 (-28.64, 153.41),
 (-30.99, 151.59),
 (-33.39, 148.01),
 (-34.75, 149.73),
 (-35.28, 149.12),
 (-33.6968, 150.5685),
 (-28.5207, 153.5401),
 (-28.52, 153.54),
 (-30.32, 153.12),
 (-30.32, 153.12),
 (-29.49, 153.24),
 (-35.23, 149.01),
 (-33.42, 149.59),
 (-29.5, 151.3),
 (-33.75, 150.68),
 (-33.84, 150.85),
 (-33.64, 151.33),
 (-33.69, 151.1096),
 (-33.74, 151.13),
 (-33.74, 151.13),
 (-33.28, 151.57),
 (-33.43, 151.34),
 (-28.65, 153.62),
 (-32.782, 152.085),
 (-33.729, 150.954),
 (-35.32, 149.12),
 (-32.77, 151.08),
 (-33.6, 149.4),
 (-30.22, 151.67),
 (-30.215, 151.668),
 (-32.265, 150.885),
 (-28.39, 153.19),
 (-33.89, 150.91),
 (-33.865, 150.951),
 (-30.441, 151.353),
 (-29.932, 152.718),
 (-35.36, 149.23),
 (-34.74, 148.89),
 (-33.697, 150.568),
 (-31.45, 142.13),
 (-31.43, 147.27),
 (-33.56, 150.218),
 (-31.45, 142.13),
 (-31.15, 150.985),
 (-32.97, 151.7),
 (-33.42, 149.58),
 (-32.099, 152.388),
 (-31.432, 152.485),
 (-31.18, 151.43),
 (-33.42, 149.58),
 (-3